# Cloud Workshop Azure Databricks
## 07. Classification Evaluation

## Evaluating a Classification Model

In this exercise, you will create a pipeline for a classification model, and then apply commonly used metrics to evaluate the resulting classifier.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(data)

# Split the data
splits = data.randomSplit([0.7, 0.3])

train = splits[0]
test = splits[1]

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
21,2,WN,10721,13342,26,57,1
13,1,AA,15016,12892,51,27,1
5,5,FL,10397,11433,9,4,0
22,1,US,11278,14100,35,71,1
23,4,WN,12451,10693,9,5,0
5,7,AA,11298,15016,39,42,1
4,6,UA,13930,14307,71,58,1
10,3,9E,14307,11433,68,140,1
29,7,UA,14057,14771,130,125,1
14,7,UA,14771,11292,20,42,1


### Define the Pipeline and Train the Model
Now define a pipeline that creates a feature vector and trains a classification model

In [5]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")
featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

lr = LogisticRegression(labelCol="Late", featuresCol="features")

pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])

model = pipeline.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [7]:
prediction = model.transform(test)
predicted = prediction.select("features", col("prediction").cast("Int"), col("Late").alias("trueLabel"))

predicted.show(100, truncate=False)

+---------------------------------------------+----------+---------+
features |prediction|trueLabel|
+---------------------------------------------+----------+---------+
[24.0,2.0,10.0,1.0,52.0,0.26865671641791045] |0 |0 |
[24.0,2.0,10.0,57.0,33.0,0.3582089552238806] |1 |1 |
[24.0,2.0,10.0,11.0,12.0,0.38308457711442784]|1 |1 |
[24.0,2.0,10.0,11.0,12.0,0.4427860696517413] |1 |1 |
[24.0,2.0,10.0,38.0,54.0,0.3383084577114428] |0 |1 |
[24.0,2.0,10.0,38.0,22.0,0.24378109452736318]|0 |1 |
[24.0,2.0,10.0,38.0,49.0,0.2736318407960199] |0 |0 |
[24.0,2.0,10.0,38.0,42.0,0.24378109452736318]|0 |0 |
[24.0,2.0,10.0,38.0,1.0,0.22885572139303484] |0 |0 |
[24.0,2.0,10.0,14.0,63.0,0.26865671641791045]|0 |1 |
[24.0,2.0,10.0,12.0,19.0,0.40298507462686567]|1 |0 |
[24.0,2.0,10.0,12.0,19.0,0.47761194029850745]|1 |1 |
[24.0,2.0,10.0,12.0,25.0,0.900497512437811] |1 |1 |
[24.0,2.0,10.0,12.0,1.0,0.2537313432835821] |0 |1 |
[24.0,2.0,10.0,17.0,38.0,0.582089552238806] |1 |1 |
[24.0,2.0,10.0,0.0,16.0,0.5522388059701493] |1 |1 |
[24.0,2.0,10.0,64.0,12.0,0.5671641791044776] |1 |1 |
[24.0,2.0,2.0,34.0,5.0,0.34328358208955223] |1 |0 |
[24.0,2.0,2.0,27.0,2.0,0.2537313432835821] |0 |1 |
[24.0,2.0,2.0,11.0,24.0,0.3482587064676617] |1 |1 |
[24.0,2.0,2.0,35.0,5.0,0.2537313432835821] |0 |0 |
[24.0,2.0,2.0,2.0,32.0,0.6467661691542289] |1 |1 |
[24.0,2.0,2.0,2.0,9.0,0.25870646766169153] |0 |0 |
[24.0,2.0,2.0,2.0,9.0,0.4975124378109453] |1 |1 |
[24.0,2.0,2.0,2.0,22.0,0.24875621890547264] |0 |0 |
[24.0,2.0,2.0,2.0,22.0,0.472636815920398] |1 |1 |
[24.0,2.0,2.0,2.0,11.0,0.29850746268656714] |0 |0 |
[24.0,2.0,2.0,2.0,2.0,0.263681592039801] |0 |0 |
[24.0,2.0,2.0,2.0,14.0,0.25870646766169153] |0 |0 |
[24.0,2.0,2.0,2.0,36.0,0.2537313432835821] |0 |0 |
[24.0,2.0,2.0,2.0,52.0,0.26865671641791045] |0 |0 |
[24.0,2.0,2.0,2.0,59.0,0.582089552238806] |1 |1 |
[24.0,2.0,2.0,2.0,34.0,0.2736318407960199] |0 |0 |
[24.0,2.0,2.0,2.0,38.0,0.23880597014925373] |0 |0 |
[24.0,2.0,2.0,2.0,38.0,0.3681592039800995] |1 |0 |
[24.0,2.0,2.0,2.0,41.0,0.2885572139303483] |0 |0 |
[24.0,2.0,2.0,2.0,28.0,0.5024875621890548] |1 |1 |
[24.0,2.0,2.0,14.0,5.0,0.681592039800995] |1 |1 |
[24.0,2.0,2.0,51.0,5.0,0.23383084577114427] |0 |0 |
[24.0,2.0,2.0,3.0,5.0,0.5074626865671642] |1 |1 |
[24.0,2.0,2.0,3.0,7.0,0.5870646766169154] |1 |1 |
[24.0,2.0,2.0,3.0,1.0,0.23880597014925373] |0 |1 |
[24.0,2.0,2.0,13.0,5.0,0.572139303482587] |1 |1 |
[24.0,2.0,2.0,13.0,24.0,0.2537313432835821] |0 |1 |
[24.0,2.0,2.0,13.0,24.0,0.48756218905472637] |1 |1 |
[24.0,2.0,2.0,15.0,5.0,0.24875621890547264] |0 |0 |
[24.0,2.0,2.0,21.0,4.0,0.31343283582089554] |0 |0 |
[24.0,2.0,2.0,21.0,5.0,0.34328358208955223] |1 |1 |
[24.0,2.0,2.0,21.0,2.0,0.29850746268656714] |0 |1 |
[24.0,2.0,2.0,21.0,28.0,0.43781094527363185] |1 |1 |
[24.0,2.0,2.0,36.0,24.0,0.24875621890547264] |0 |1 |
[24.0,2.0,2.0,54.0,5.0,0.21393034825870647] |0 |0 |
[24.0,2.0,2.0,0.0,32.0,0.21890547263681592] |0 |0 |
[24.0,2.0,2.0,0.0,22.0,0.5373134328358209] |1 |1 |
[24.0,2.0,2.0,0.0,5.0,0.23880597014925373] |0 |0 |
[24.0,2.0,2.0,0.0,13.0,0.22388059701492538] |0 |1 |
[24.0,2.0,2.0,0.0,13.0,0.736318407960199] |1 |1 |
[24.0,2.0,2.0,0.0,29.0,0.39303482587064675] |1 |1 |
[24.0,2.0,2.0,63.0,5.0,0.5522388059701493] |1 |1 |
[24.0,2.0,2.0,5.0,5.0,0.5074626865671642] |1 |1 |
[24.0,2.0,2.0,5.0,1.0,0.3383084577114428] |1 |1 |
[24.0,2.0,2.0,28.0,5.0,0.6268656716417911] |1 |1 |
[24.0,2.0,12.0,69.0,17.0,0.26865671641791045]|0 |0 |
[24.0,2.0,12.0,8.0,34.0,0.835820895522388] |1 |1 |
[24.0,2.0,12.0,3.0,34.0,0.7661691542288557] |1 |1 |
[24.0,2.0,12.0,23.0,17.0,0.417910447761194] |1 |1 |
[24.0,2.0,12.0,5.0,17.0,0.2537313432835821] |0 |0 |
[24.0,2.0,12.0,40.0,17.0,0.24875621890547264]|0 |0 |
[24.0,2.0,7.0,11.0,26.0,0.2935323383084577] |0 |1 |
[24.0,2.0,7.0,11.0,25.0,0.6268656716417911] |1 |1 |
[24.0,2.0,7.0,11.0,14.0,0.2537313432835821] |0 |1 |
[24.0,2.0,7.0,11.0,1.0,0.5422885572139303] |1 |1 |
[24.0,2.0,7.0,11.0,33.0,0.6069651741293532] |1 |1 |
[24.0,2.0,7.0,11.0,28.0,0.4079601990049751] |1 |1 |
[24

### Compute Confusion Matrix Metrics
Classifiers are typically evaluated by creating a *confusion matrix*, which indicates the number of:
- True Positives
- True Negatives
- False Positives
- False Negatives

From these core measures, other evaluation metrics such as *precision* and *recall* can be calculated.

In [9]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())

metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])


In [10]:
display(metrics)

metric,value
TP,88954.0
FP,7352.0
TN,81776.0
FN,13253.0
Precision,0.9236600004153428
Recall,0.8703317776668917


### View the Raw Prediction and Probability
The prediction is based on a raw prediction score that describes a labelled point in a logistic function. This raw prediction is then converted to a predicted label of 0 or 1 based on a probability vector that indicates the confidence for each possible label value (in this case, 0 and 1). The value with the highest confidence is selected as the prediction.

In [12]:
prediction.select("rawPrediction", "probability", "prediction", col("Late").alias("trueLabel")).show(100, truncate=False)

+------------------------------------------+------------------------------------------+----------+---------+
rawPrediction |probability |prediction|trueLabel|
+------------------------------------------+------------------------------------------+----------+---------+
[1.7134334694716848,-1.7134334694716848] |[0.8472810908559336,0.15271890914406644] |0.0 |0 |
[-0.45003076139702713,0.45003076139702713]|[0.3893534522779078,0.6106465477220921] |1.0 |1 |
[-1.5785221451936575,1.5785221451936575] |[0.17100488414041817,0.8289951158595819] |1.0 |1 |
[-3.1078581872551982,3.1078581872551982] |[0.04278427369899664,0.9572157263010033] |1.0 |1 |
[0.1681364928798974,-0.1681364928798974] |[0.5419353773947365,0.45806462260526354] |0.0 |1 |
[2.253879039806863,-2.253879039806863] |[0.9049846083022937,0.09501539169770624] |0.0 |1 |
[1.7724631134980564,-1.7724631134980564] |[0.8547637159119438,0.14523628408805617] |0.0 |0 |
[2.4636954062675755,-2.4636954062675755] |[0.9215572184440256,0.0784427815559744] |0.0 |0 |
[2.415905865538499,-2.415905865538499] |[0.9180321914590586,0.08196780854094131] |0.0 |0 |
[1.9054083628946596,-1.9054083628946596] |[0.8705024219597854,0.12949757804021447] |0.0 |1 |
[-2.008974644398851,2.008974644398851] |[0.11826385750221301,0.881736142497787] |1.0 |0 |
[-3.9206446969757778,3.9206446969757778] |[0.019442789761446134,0.9805572102385538] |1.0 |1 |
[-14.690496751640145,14.690496751640145] |[4.1686760127172984E-7,0.9999995831323988]|1.0 |1 |
[1.6255307309403593,-1.6255307309403593] |[0.8355564735056802,0.16444352649431979] |0.0 |1 |
[-6.368204956342112,6.368204956342112] |[0.0017122983914272666,0.9982877016085727]|1.0 |1 |
[-5.934472643170658,5.934472643170658] |[0.002639632000888505,0.9973603679991115] |1.0 |1 |
[-5.98178092085101,5.98178092085101] |[0.0025179704806747426,0.9974820295193252]|1.0 |1 |
[-0.07486086507919332,0.07486086507919332]|[0.4812935190733842,0.5187064809266159] |1.0 |0 |
[2.1464375704988523,-2.1464375704988523] |[0.895335411774055,0.104664588225945] |0.0 |1 |
[-0.1384604111902643,0.1384604111902643] |[0.4654400927227076,0.5345599072772923] |1.0 |1 |
[2.225033651233856,-2.225033651233856] |[0.9024751247714107,0.09752487522858927] |0.0 |0 |
[-7.7542281539003195,7.7542281539003195] |[4.287412417586312E-4,0.9995712587582414] |1.0 |1 |
[1.9451672980698804,-1.9451672980698804] |[0.8749187280373466,0.12508127196265345] |0.0 |0 |
[-4.172176870176285,4.172176870176285] |[0.01518453399154445,0.9848154660084556] |1.0 |1 |
[2.3364372766129327,-2.3364372766129327] |[0.9118501352460987,0.08814986475390126] |0.0 |0 |
[-3.398572881117845,3.398572881117845] |[0.03234009546728295,0.967659904532717] |1.0 |1 |
[0.9465915733415908,-0.9465915733415908] |[0.7204291995615223,0.2795708004384777] |0.0 |0 |
[1.7442868996368341,-1.7442868996368341] |[0.8512307631765124,0.14876923682348767] |0.0 |0 |
[1.997621389685058,-1.997621389685058] |[0.8805471128601239,0.11945288713987616] |0.0 |0 |
[2.3558640629636365,-2.3558640629636365] |[0.9133992072347928,0.08660079276520716] |0.0 |0 |
[2.141383145616822,-2.141383145616822] |[0.8948608147359901,0.10513918526400987] |0.0 |0 |
[-5.81419534694502,5.81419534694502] |[0.0029759981543569266,0.9970240018456431]|1.0 |1 |
[1.8251037456303862,-1.8251037456303862] |[0.8611774097987112,0.13882259020128884] |0.0 |0 |
[2.7591797101250934,-2.7591797101250934] |[0.9404296966035396,0.05957030339646033] |0.0 |0 |
[-0.554381714341579,0.554381714341579] |[0.3648484165403725,0.6351515834596275] |1.0 |0 |
[1.5162054633762487,-1.5162054633762487] |[0.8199790360108618,0.18002096398913833] |0.0 |0 |
[-4.100295992210404,4.100295992210404] |[0.016297753304438764,0.9837022466955613] |1.0 |1 |
[-8.858907501176,8.858907501176] |[1.4209004220078493E-4,0.9998579099577993]|1.0 |1 |
[2.82905958345283,-2.82905958345283] |[0.9442260974353275,0.05577390256467261] |0.0 |0 |
[-4.463139030591281,4.463139030591281] |[0.011394787825350824,0.9886052121746491] |1.0 |1 |
[-6.4812721166939316,6.4812721166939316] |[0.001529517725

Note that the results include rows where the probability for 0 (the first value in the **probability** vector) is only slightly higher than the probability for 1 (the second value in the **probability** vector). The default *discrimination threshold* (the boundary that decides whether a probability is predicted as a 1 or a 0) is set to 0.5; so the prediction with the highest probability is always used, no matter how close to the threshold.

### Review the Area Under ROC
Another way to assess the performance of a classification model is to measure the area under a *received operator characteristic (ROC) curve* for the model. The **spark.ml** library includes a **BinaryClassificationEvaluator** class that you can use to compute this. A ROC curve plots the True Positive and False Positive rates for varying threshold values (the probability value over which a class label is predicted). The area under this curve gives an overall indication of the models accuracy as a value between 0 and 1. A value under 0.5 means that a binary classification model (which predicts one of two possible labels) is no better at predicting the right class than a random 50/50 guess.

In [15]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(prediction)

print ("AUC = ", auc)

AUC = 0.9480805966133202

### Review the AreaUnderPR

In [17]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderPR")
areaUnderPR = evaluator.evaluate(prediction)
print ("areaUnderPR = ", areaUnderPR)

areaUnderPR = 0.9640417883303767

> Fin